In [21]:
import genjax
import rich
from rich import inspect
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import pyarrow as pa
import pandas as pd
import numpy as np

console = genjax.pretty()

In [102]:
@genjax.gen
def model(key, x):
    key, y = genjax.trace("x", genjax.Normal, shape=(2, 2))(key, (x, 1.0))
    return key, y

In [103]:
key = jax.random.PRNGKey(314159)
key, *sub_keys = jax.random.split(key, 100 + 1)
sub_keys = jnp.array(sub_keys)
_, tr = jax.jit(jax.vmap(model.simulate, in_axes=(0, None)))(sub_keys, (5.0,))
tr

BuiltinTrace
└── x
    └── DistributionTrace
        ├── gen_fn
        │   └── _Normal
        ├── args
        │   └── (f32[100], f32[100])
        ├── value
        │   └── ValueChoiceMap
        │       └── value
        │           └── f32[100,2,2]
        └── score
            └── f32[100]

In [117]:
def get_dataframe(tr: genjax.Trace):
    values, forms = jtu.tree_flatten(tr)

    # This forces scalars into a single-dim array.
    def _check_expand(v):
        if v.shape == ():
            return np.expand_dims(v, axis=-1)
        else:
            return v

    arrays = list(map(lambda v: _check_expand(np.array(v)), values))
    df = pd.DataFrame(arrays).T
    df.columns = df.columns.astype(str)
    return df

In [121]:
df = get_dataframe(tr)
df.to_feather("scratch")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_24124/1081730457.py:2 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_24124/1081730457.py'                 │
│                                                                                                  │
│ /Users/mccoybecker/Library/Caches/pypoetry/virtualenvs/genjax-BuuDBHOj-py3.10/lib/python3.10/sit │
│ e-packages/pandas/util/_decorators.py:211 in wrapper                                             │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                                                            │
│                                                                                                  │
│ /Users/mccoybecker/Library/Caches/pypoetry/virtualenvs/genjax-BuuDBHOj-py3.10/lib/python3.10/sit │
│ e-packages/pandas/core/frame.py:2794 in to_feather                                               │
│                                                                                                  │
│    2791 │   │   """                                                                              │
│    2792 │   │   from pandas.io.feather_format import to_feather                                  │
│    2793 │   │                                                                                    │
│ ❱  2794 │   │   to_feather(self, path, **kwargs)                                                 │
│    2795 │                                                                                        │
│    2796 │   @doc(                                                                                │
│    2797 │   │   Series.to_markdown,                                                              │
│                                                                                                  │
│ /Users/mccoybecker/Library/Caches/pypoetry/virtualenvs/genjax-BuuDBHOj-py3.10/lib/python3.10/sit │
│ e-packages/pandas/io/feather_format.py:93 in to_feather                                          │
│                                                                                                  │
│    90 │   with get_handle(                                                                       │
│    91 │   │   path, "wb", storage_options=storage_options, is_text=False                         │
│    92 │   ) as handles:                                                                          │
│ ❱  93 │   │   feather.write_feather(df, handles.handle, **kwargs)                                │
│    94                                                                                            │
│    95                                                                                            │
│    96 @doc(storage_options=_shared_docs["storage_options"])                                      │
│                                                                                                  │
│ /Users/mccoybecker/Library/Caches/pypoetry/virtualenvs/genj